In [1]:
import cv2
import numpy as np
import os
import pandas as pd
import csv

from sklearn.cluster import MiniBatchKMeans
# from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
from rembg import remove
import glob
import joblib
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn import metrics
from commonFunctions import *

## Extract keypoints from each image

In [ ]:
img_path = '../input/images/'
train = pd.read_csv('../input/train.csv')
species = train.species.sort_values().unique()

dico = []

def step1():
    for leaf in train.id:
        img = cv2.imread(img_path + str(leaf) + ".jpg")
        kp, des = cv2.sift.detectAndCompute(img, None)

        for d in des:
            dico.append(d)

In [17]:
arr = ["0","1","2","3","4","5"]
dico = []
lower_blue = np.array([0, 0, 65])
upper_blue = np.array([20, 255, 255])


for label in arr:
    dirList = glob.glob("../Dataset_0-5/men/"+str(label)+"/*.jpg")
    
    for img_path in dirList:
        try:
            im= cv2.imread(img_path)
            img = cv2.resize(im, (512,512))
            img_hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
            mask = cv2.inRange(img_hls, lower_blue, upper_blue)
            result = cv2.bitwise_and(img, img, mask=mask)
            
            
            sift = cv2.xfeatures2d.SIFT_create()
            kp, des = sift.detectAndCompute(result, None)
            for d in des:
                dico.append(d)
        except:
            pass
        

In [18]:
import numpy as np
import cv2

# Load image and detect SIFT keypoints and descriptors
gray = cv2.imread('test.png', 0)
sift = cv2.xfeatures2d.SIFT_create()
kp, des = sift.detectAndCompute(gray, None)

# Stack descriptors into a NumPy array
# des_array = np.vstack(des)
dd = []
for d in des:
    dd.append(d)
# print(np.shape(des_array))
print(np.shape(dd))


(598, 128)


In [35]:
img = cv2.imread('test.png')
gray= cv2.cvtColor(img,cv.COLOR_BGR2GRAY)
sift = cv2.SIFT_create()
kp = sift.detect(gray,None)
des = sift.compute(gray,kp)
des = np.array(des)

# desc_samples = des[np. random. randint(len(des), size = 20)]
# print(desc_samples)
print(np.shape(des))
img=cv2.drawKeypoints(gray,kp,img)
# cv2.imwrite('sift_keypoints.jpg',img)
# show_images([img])

C:\Users\khale\AppData\Local\Temp\ipykernel_3948\479942303.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  des = np.array(des)


ValueError: could not broadcast input array from shape (608,128) into shape (608,)

## Clustering
We now have an array with a huge number of descriptors. We cannot use all of them to create or model so we need to cluster them. A rule-of-thumb is to create k centers with k = number of categories * 10 (in our case, it's 60).

In [ ]:
def step2():
    k = np.size(species) * 10

    batch_size = np.size(os.listdir(img_path)) * 3
    kmeans = MiniBatchKMeans(n_clusters=k, batch_size=batch_size, verbose=1).fit(dico)
    
# I use MiniBatchKMeans to avoid Memory Error.

In [40]:
print(np.shape(dico))

(102875, 128)


In [9]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=60)
kmeans.fit(dico)
joblib.dump(kmeans, "model/Tkmeans.pkl")

c:\Users\khale\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


['model/Tkmeans.pkl']

In [19]:
k = 10 * 6 
batch_size = 50
# kmeans = MiniBatchKMeans(n_clusters=k, batch_size=batch_size, verbose=1).fit(dico)
kmeans = MiniBatchKMeans(n_clusters=k, batch_size=batch_size, verbose=1, max_iter=500).fit(dico)

joblib.dump(kmeans, "model/kmeans.pkl")

c:\Users\khale\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Init 1/3 with method k-means++
Inertia for init 1/3: 18774563.0
Init 2/3 with method k-means++
Inertia for init 2/3: 19165245.0
Init 3/3 with method k-means++
Inertia for init 3/3: 19357251.0
[MiniBatchKMeans] Reassigning 25 cluster centers.
Minibatch step 1/2858540: mean batch inertia: 123581.4
[MiniBatchKMeans] Reassigning 22 cluster centers.
Minibatch step 2/2858540: mean batch inertia: 115719.81207199547, ewa inertia: 115719.81207199547
Minibatch step 3/2858540: mean batch inertia: 96467.25802568655, ewa inertia: 115713.07699510464
Minibatch step 4/2858540: mean batch inertia: 108905.30418700691, ewa inertia: 115710.69544753399
Minibatch step 5/2858540: mean batch inertia: 96073.60621063113, ewa inertia: 115703.82584957806
Minibatch step 6/2858540: mean batch inertia: 95648.357723694, ewa inertia: 115696.80989109355
Minibatch step 7/2858540: mean batch inertia: 96115.95322390068, ewa inertia: 115689.9599648487
Minibatch step 8/2858540: mean batch inertia: 89646.58787806866, ewa ine

['model/kmeans.pkl']

## Creation of the histograms
To create our each image by a histogram. We will create a vector of k value for each image. For each keypoints in an image, we will find the nearest center and increase by one its value.

In [ ]:
def step3():
    kmeans.verbose = False

    histo_list = []

    for leaf in train.id:
        img = cv2.imread(img_path + str(leaf) + ".jpg")
        kp, des = sift.detectAndCompute(img, None)

        histo = np.zeros(k)
        nkp = np.size(kp)

        for d in des:
            idx = kmeans.predict([d])
            histo[idx] += 1/nkp # Because we need normalized histograms, I prefere to add 1/nkp directly

        histo_list.append(histo)

In [20]:
# kmeans.verbose = False
# kmeans=joblib.load("model/kmeans.pkl")
histo_list = []
Y = []
arr = ["0","1","2","3","4","5"]

lower_blue = np.array([0, 0, 65])
upper_blue = np.array([20, 255, 255])


for label in arr:
    dirList = glob.glob("../Dataset_0-5/men/"+str(label)+"/*.jpg")
    
    for img_path in dirList:
        try:
            im= cv2.imread(img_path)
            img = cv2.resize(im, (512,512))
            img_hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
            mask = cv2.inRange(img_hls, lower_blue, upper_blue)
            result = cv2.bitwise_and(img, img, mask=mask)
            
            
            sift = cv2.xfeatures2d.SIFT_create()
            kp, des = sift.detectAndCompute(result, None)
            histo = np.zeros(60)
            nkp = np.size(kp)
            for d in des:
                idx = kmeans.predict([d])
                histo[idx] += 1/nkp # Because we need normalized histograms, I prefere to add 1/nkp directly

            histo_list.append(histo)
            Y.append(label)
        except:
            pass
        

## train our model

In [13]:
print(np.shape(histo_list))

(1047, 60)


In [21]:
X = np.array(histo_list)
train_x,test_x,train_y,test_y = train_test_split(histo_list,Y, test_size = 0.3)
classifier=SVC(kernel="linear", random_state=6)
classifier.fit(train_x,train_y)
joblib.dump(classifier, "model/sift_model.pkl")

['model/sift_model.pkl']

In [22]:
from sklearn import metrics

prediction=classifier.predict(test_x)
print("Accuracy= ",metrics.accuracy_score(prediction, test_y))
print("Precision= ",metrics.precision_score(prediction, test_y, average='weighted'))
print("Recall= ",metrics.recall_score(prediction, test_y, average='weighted'))
print("F1 Score= ",metrics.f1_score(prediction, test_y, average='weighted'))
print("Confusion Matrix= ",metrics.confusion_matrix(prediction, test_y))
print("Classification Report= ",metrics.classification_report(prediction, test_y))

Accuracy=  0.1365079365079365
Precision=  1.0
Recall=  0.1365079365079365
F1 Score=  0.24022346368715086
Confusion Matrix=  [[43 59 49 57 53 54]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]
Classification Report=                precision    recall  f1-score   support

           0       1.00      0.14      0.24       315
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0

    accuracy                           0.14       315
   macro avg       0.17      0.02      0.04       315
weighted avg       1.00      0.14      0.24       315



c:\Users\khale\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\khale\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\khale\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:

In [23]:
from sklearn.ensemble import RandomForestClassifier

# Create a random forest classifier and fit it to the training data
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(train_x, train_y)


RandomForestClassifier()

In [24]:
prediction=rfc.predict(test_x)
print("Accuracy= ",metrics.accuracy_score(prediction, test_y))
print("Precision= ",metrics.precision_score(prediction, test_y, average='weighted'))
print("Recall= ",metrics.recall_score(prediction, test_y, average='weighted'))
print("F1 Score= ",metrics.f1_score(prediction, test_y, average='weighted'))
print("Confusion Matrix= ",metrics.confusion_matrix(prediction, test_y))
print("Classification Report= ",metrics.classification_report(prediction, test_y))

Accuracy=  0.35873015873015873
Precision=  0.3921433850756354
Recall=  0.35873015873015873
F1 Score=  0.3660398898057634
Confusion Matrix=  [[24 14  9 14  7  3]
 [ 1 22  8  5  4  2]
 [ 2  9 12  8  6  2]
 [ 9  3 11 11  9  8]
 [ 5  4  2 15 13  8]
 [ 2  7  7  4 14 31]]
Classification Report=                precision    recall  f1-score   support

           0       0.56      0.34      0.42        71
           1       0.37      0.52      0.44        42
           2       0.24      0.31      0.27        39
           3       0.19      0.22      0.20        51
           4       0.25      0.28      0.26        47
           5       0.57      0.48      0.52        65

    accuracy                           0.36       315
   macro avg       0.36      0.36      0.35       315
weighted avg       0.39      0.36      0.37       315



In [ ]:
def step4():
    X = np.array(histo_list)
    Y = []

    # It's a way to convert species name into an integer
    for s in train.species:
        Y.append(np.min(np.nonzero(species == s)))

    mlp = MLPClassifier(verbose=True, max_iter=600000)
    mlp.fit(X, Y)

I also run this script with ORB instead of SIFT and I got best results. To do it, just replace `cv2.xfeatures2d.SIFT_create()` by `cv2.ORB_create().`